# Ej 6

El porcentaje de contenidos que están publicados cuya última edición no tiene comentario para los usuarios que realizaron 1, >10 y >100 de las últimas ediciones.

In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

In [2]:
# Cargo los datasets
wiki_contents=pd.read_csv('./Wikipedia_dataset_2c2021/contents.csv', \
                          usecols=['revisor_id','revisor_ip','revisor_comment'])

# 1. Creo un identificador unico para cada uno de los usuarios

In [3]:
wiki_contents.revisor_id.fillna(wiki_contents.revisor_ip,inplace=True)
del wiki_contents["revisor_ip"]

In [4]:
wiki_contents=wiki_contents.rename(columns={"revisor_id":"identificador_editor", \
                                            "revisor_comment":"comentario_editor"})

wiki_contents

,identificador_editor,comentario_editor
0,"1,708,233.00",Así esta mejor
1,2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2,Añadir contenido
2,"4,642,283.00",Reemplazos con [[Usuario:Benjavalero/Replacer|...
3,"5,338,508.00",Reemplazos con [[Usuario:Benjavalero/Replacer|...
4,"3,615,366.00",Correcciones ortográficas con [[Usuario:Benjav...
...,...,...
4132159,"474,907.00",termino traducción y quito plantilla en desarr...
4132160,"4,032,712.00",Crls9011 trasladó la página [[Manifestación de...
4132161,"158,577.00",Gabriel Solo trasladó la página [[Rocasaurus]]...
4132162,"5,429,951.00",Términus trasladó la página [[Medalla Conmemor...


# 2. Evaluo si hay datos invalidos y los borro en caso de haber

In [5]:
wiki_contents["identificador_editor"].hasnans

True

In [6]:
# Borro aquellos identificadores que no tienen revisor_username, revisor_id p revisor_ip
# Estos los considero como datos invalidos

wiki_contents.dropna(subset=['identificador_editor'], inplace=True)

In [7]:
wiki_contents["identificador_editor"].hasnans

False

# 3. Cuento la cantidad de comentarios validos e invalidos por usuario

In [8]:
datos_usuarios = wiki_contents.groupby('identificador_editor').agg(total_comentarios=('comentario_editor', 'size'),
                                                           comentarios_validos=('comentario_editor','count') )

In [9]:
datos_usuarios

,total_comentarios,comentarios_validos
identificador_editor,,
0.0,7,7
1.0,93,43
5.0,10,10
7.0,27,19
10.0,1,1
...,...,...
99.253.44.18,1,1
99.29.229.61,1,1
99.4.169.60,1,1


# 4. Categorizo cada usuario segun la cantidad de comentarios hechos

In [10]:
def definir_tipo_usuario(cant_commentarios):
    if cant_commentarios == 1:
        return 'principiante'
    elif cant_commentarios>10 and cant_commentarios<=100:
        return 'amateur'
    elif cant_commentarios > 100:
        return 'pro'
    else:
        return np.nan

In [11]:
datos_usuarios['categoria'] = datos_usuarios[('total_comentarios')].transform(definir_tipo_usuario)

In [12]:
datos_usuarios

,total_comentarios,comentarios_validos,categoria
identificador_editor,,,
0.0,7,7,NaN
1.0,93,43,amateur
5.0,10,10,NaN
7.0,27,19,amateur
10.0,1,1,principiante
...,...,...,...
99.253.44.18,1,1,principiante
99.29.229.61,1,1,principiante
99.4.169.60,1,1,principiante


# 5. Calculo el porcentaje de comentarios invaldios por usuario

In [13]:
datos_usuarios['%_comentarios_invalidos']= ( 
(datos_usuarios['total_comentarios'] - datos_usuarios['comentarios_validos']) / datos_usuarios['total_comentarios']
    )*100

In [14]:
datos_usuarios

,total_comentarios,comentarios_validos,categoria,%_comentarios_invalidos
identificador_editor,,,,
0.0,7,7,NaN,0.00
1.0,93,43,amateur,53.76
5.0,10,10,NaN,0.00
7.0,27,19,amateur,29.63
10.0,1,1,principiante,0.00
...,...,...,...,...
99.253.44.18,1,1,principiante,0.00
99.29.229.61,1,1,principiante,0.00
99.4.169.60,1,1,principiante,0.00


# 5. Creo la estadistica segun los tipos de usuarios

Esta estadistica me dice el porcentaje de contenido publicado cuya ultima edicion no tiene comentario, para cada tipo de usuario (principiante, amateur y pro)

In [15]:
estadisticas = datos_usuarios.groupby('categoria').agg( porcentaje=('%_comentarios_invalidos','mean'), \
                                  muestra = ('%_comentarios_invalidos', 'count'))

In [16]:
estadisticas.sort_values('porcentaje', ascending=False)

,porcentaje,muestra
categoria,,
principiante,35.22,114721
amateur,27.53,5884
pro,20.17,1894


# Conclusion

Se puede observar que cerca del 35% de los usuarios principiantes dejan comentarios invalidos (vacios). 

Esto es entendible ya que posiblemente aun no conocen las normas de la comunidad de Wikipedia.

Se ve que en promedio, mientras mas practica tienen los usuarios van asentando el habito de dejar comentarios en las ediciones y ya cuando llegan al nivel profesional; cerca de un 20%, para una muestra de 1894 editores, editan sin dejar comentarios.